In [2]:
import galois
import numpy as np
from os.path import exists

## Pre-Computed Inverse Vandermonde Matrix

From the [previous notebook](https://github.com/Blake-Haydon/Polynomial-Interpolation/blob/main/matrix_interpolation.ipynb) we know that the inverse Vandermonde matrix can be used to find the coefficients of an interpolated polynomial. To find the coefficients we need to solve the following equation:

$$\bf{y}=V(\bf{x})\bf{a}$$

Therefore: 

$$\bf{a}=V(\bf{x})^{-1}\bf{y}$$

### Pre-Computation Trick 

Ordinarily we would need to compute the inverse of the Vandermonde matrix for each set of input values $\bf{x}$. However, if we fix the input values $\bf{x}$ we can pre-compute the inverse Vandermonde matrix and use it to find the coefficients for any set of output values $\bf{y}$. This is useful if we want to interpolate a large number of values with the same input values.

### Example

Let's say we want to interpolate the values $y_1=1$, $y_2=4$, $y_3=2$ and $y_4=6$ with the input values $x_1=1$, $x_2=2$, $x_3=3$ and $x_4=4$. We can pre-compute the inverse Vandermonde matrix for these input values and use it to find the coefficients for any set of output values.

First we need to compute the inverse Vandermonde matrix:
<!-- V_inv = np.linalg.inv(np.vander([1, 2, 3, 4], 4)) -->

$$V^{-1}=\begin{bmatrix}-0.16666667& 0.5&-0.5&0.16666667\\1.5&-4&3.5&-1\\-4.3&9.5&-7&1.83\\4&-6&4&-1\end{bmatrix}$$

Then we can find the coefficients for any set of output values:
<!-- a = V_inv @ y -->

$$\bf{a}=V^{-1}\bf{y}=\begin{bmatrix}1.83\\-13.5\\30.6\\-18\end{bmatrix}$$

In [11]:
# Example code for precomputing the inverse of the Vandermonde matrix
V = np.vander([1, 2, 3, 4])
V_inv = np.linalg.inv(V)
y = np.array([1, 4, 2, 6])
a = V_inv @ y

# Find value at x=2, should be 4
np.polyval(a, 2)

4.0

In [4]:
def save_vandermonde_inv(prime: int, size: int):
    # Create Galois field
    GF = galois.GF(prime)
    a = GF.primitive_element

    # Create Vandermonde matrix
    V = GF.Vandermonde(a, size, size)
    V = np.flip(V, axis=1)

    # Invert Vandermonde matrix and save to file
    filename = f'vandermonde_inverses/V_inv_F_{prime}_size_{size}.npz'
    np.savez_compressed(filename, V_inv=np.linalg.inv(V))

def load_vandermonde_inv(prime: int, size: int):
    # Create Galois field
    GF = galois.GF(prime)
    
    # Load inverse Vandermonde matrix from file
    filename = f'vandermonde_inverses/V_inv_F_{prime}_size_{size}.npz'
    V_inv = np.load(filename)['V_inv']
    return GF(V_inv)

In [5]:
def fast_poly(y: galois.FieldArray, prime: int, size: int):
    # Load inverse Vandermonde matrix from file
    V_inv = load_vandermonde_inv(prime, size)
    
    # Compute polynomial coefficients
    coeffs = np.matmul(V_inv, y)
    return galois.Poly(coeffs)

def eval_poly(poly: galois.Poly, x: int):
    # Because the polynomial is generated from a Vandermonde matrix with a primitive element as the base of 
    # the exponent we must evaluate slightly differently.
    return poly(poly.field.primitive_element**x)

In [6]:
#### CONSTANTS ####
# The following constants are already generated and therefore are very fast to compute
prime = 5003 # prime field size
size = 5000  # max degree of polynomial

GF = galois.GF(prime)

In [7]:
# Generation is very slow for large sizes (only need to do once)
if not exists(f"vandermonde_inverses/V_inv_F_{prime}_size_{size}.npz"):
    save_vandermonde_inv(prime, size)

In [8]:
y = GF.Random(size) # Random values to interpolate
poly = fast_poly(y, prime, size)

# Sanity check 100th element
i = 100
print("From original random `y` input:", y[i])
print("From interpolated polynomial:", eval_poly(poly, i))

From original random `y` input: 3027
From interpolated polynomial: 3027
